# Week 03 Live Coding Demo: Flow Control, Imports, and Programming Foundations
Topics: if/elif/else; for & while loops; recursion; modules & packages; package discovery; Python bytecode and .pyc files; compiled vs interpreted (with C/assembly mockups); GIL; why NumPy is fast.

## 1. if / elif / else — piecewise physics logic

In [ ]:
# kinetic energy regime based on KE value
# KE = 0.5 * m * v^2
def ke_regime(mass_kg, speed_m_s):
    E = 0.5 * mass_kg * speed_m_s**2
    if E < 1.0:
        return "low"
    elif E < 100.0:
        return "moderate"
    else:
        return "high"
print(ke_regime(0.2, 1.0))
print(ke_regime(1.0, 10.0))
print(ke_regime(5.0, 30.0))


In [ ]:
# Fluid dynamics drag regime based on Reynolds number
# Re = (density * velocity * length) / viscosity
def drag_regime(Re):
    if Re < 1e3:
        return "laminar-ish"
    elif Re < 2e5:
        return "transitional"
    else:
        return "turbulent"
print(drag_regime(500.0))
print(drag_regime(10_000.0))
print(drag_regime(300_000.0))


In [ ]:
# Measurement quality based on reading, reference, and tolerance
def quality(reading, ref, tol):
    if reading < 0:
        return "invalid"
    elif abs(reading - ref) <= tol and (ref > 0):
        return "good"
    else:
        return "outlier"
print(quality(9.79, 9.81, 0.05))
print(quality(-0.1,  1.00, 0.05))
print(quality(1.2,   1.00, 0.05))


## 2. for Loops — counting, accumulation, and comprehensions

In [ ]:
# accumulating a sum with a for loop
N = 6
total = 0
for k in range(1, N+1):
    total = total + k
print(total)


In [ ]:
# Kepler's Third Law: T = 2π * sqrt(r^3 / GM)
# GM for Earth = 3.986004418e14 m^3/s^2
GM = 3.986004418e14
radii_m = [6.9e6, 1.0e7, 2.0e7]
periods_s = []
for r in radii_m:
    T = 2*(3.141592653589793) * (r**3 / GM)**0.5
    periods_s.append(T)
print([round(T, 1) for T in periods_s])


In [ ]:
# de Broglie wavelength of electrons accelerated through a potential V
# λ = h / sqrt(2me eV), where e is elementary charge, me is electron mass, h is Planck's constant, V is in volts
import math
e = 1.602_176_634e-19
m_e = 9.109_383_7015e-31
V = [50, 100, 500, 1000]
h = 6.626_070_15e-34
lams_m = [h / math.sqrt(2*m_e*e*U) for U in V]
print([f"{lam*1e10:.2f} Å" for lam in lams_m])


## 3. while Loops — sentinels and convergence

In [ ]:
# RC circuit discharging simulation
# v(t+dt) = v(t) - (v(t)/(R*C))*dt
V0 = 5.0
R, C = 10_000.0, 1e-6
dt = 0.0001
v, t = V0, 0.0
steps = 0
threshold = 0.1 * V0
while v > threshold:
    v = v - (v/(R*C))*dt
    t = t + dt
    steps = steps + 1
print(round(t, 4), round(v, 4), steps)


In [ ]:
# approximating e with a while loop
# e = limit as n->∞ of (1 + 1/n)^n
# stop when change is less than 1e-10
# report number of steps, final n, and value of e
n = 1
prev = 0.0
steps = 0
while True:
    val = (1 + 1/n)**n
    steps = steps + 1
    if abs(val - prev) < 1e-10:
        break
    prev = val
    n = n + 1
print(steps, n, round(val, 12))


In [ ]:
# finding the first reading above a threshold with a while loop
# readings = [0.2, 0.3, 0.29, 0.31, 0.28]
# find index of first reading >= 0.30, or -1 if none
readings = [0.2, 0.3, 0.29, 0.31, 0.28]
i = 0
found_at = None
while i < len(readings):
    if readings[i] >= 0.30:
        found_at = i
        break
    i = i + 1
else:
    found_at = -1
print(found_at)


## 4. Recursion — resistor ladder equivalent resistance

In [ ]:
# calculating the resistance of a resistor ladder network
# R_total = R + (R * R_total) / (R + R_total)
# where R is the resistance of each resistor, and R_total is the total resistance of the ladder
def parallel(Ra, Rb):
    return (Ra*Rb)/(Ra+Rb)
def ladder_R(n, R=1000.0):
    if n <= 1:
        return R
    return R + parallel(R, ladder_R(n-1, R))
for depth in [1, 2, 3, 5]:
    print(depth, round(ladder_R(depth), 4))


## 5. Imports — module, from … import, alias (new examples)

In [ ]:
from math import radians as to_rad, sin as s_sin
angle_deg = 30
print(to_rad(angle_deg))
print(round(s_sin(to_rad(30)), 3))


## 6. Tiny module you own — thermo_utils.py

In [ ]:
mod_src = '''
"""Tiny thermodynamics utilities (demo)."""
def kelvin(T_c):
    """Convert Celsius to Kelvin."""
    return T_c + 273.15
def ideal_gas_P(n_mol, T_K, V_m3, R=8.314462618):
    """Return pressure (Pa) from ideal gas law P = n R T / V."""
    return n_mol * R * T_K / V_m3
'''
with open("thermo_utils.py", "w") as f:
    _ = f.write(mod_src)
print("thermo_utils.py written")


In [ ]:
import thermo_utils
print(thermo_utils.kelvin(25.0))
print(round(thermo_utils.ideal_gas_P(0.5, 300.0, 0.01)))


## 7. Relevant packages — quick one-liners (guarded)

In [ ]:
# testing imports with error handling
def try_import(name):
    try:
        return __import__(name)
    except Exception as e:
        print(f"{name} not available: {e}")
        return None
import math, cmath, time, os
print(math.isfinite(3.14))
print(cmath.phase(1+1j))
print(round(time.perf_counter(), 3))
print(os.path.basename(os.getcwd()))
np = try_import("numpy")
if np:
    print(np.linspace(0, 1, 4))
try:
    from scipy import constants as sp_constants
    print(getattr(sp_constants, 'c'))
except Exception as e:
    print("scipy not available:", e)
pd = try_import("pandas")
if pd:
    print(pd.DataFrame({"I_A":[0.1, 0.2, 0.3]}))
try:
    import matplotlib.pyplot as plt
    plt.figure(); plt.plot([0,1,2],[0,1,0]); plt.show()
    print("matplotlib plot created")
except Exception as e:
    print("matplotlib not available:", e)


## (Optional) 8. Searching, installing, and checking packages (programmatic checks)

In [ ]:
import sys, importlib.util, importlib.metadata as md, subprocess
print(sys.executable)
print(importlib.util.find_spec('matplotlib') is not None)
for pkg in ['numpy', 'matplotlib', 'pandas', 'scipy']:
    try:
        print(pkg, md.version(pkg))
    except md.PackageNotFoundError:
        print(pkg, 'not installed')
res = subprocess.run([sys.executable, "-m", "pip", "show", "matplotlib"],
                     capture_output=True, text=True)
print("matplotlib installed?", res.returncode == 0)


## 9. Programming foundations — Python bytecode & .pyc (real artifacts)

In [ ]:
# examining bytecode of a simple function
src = """def energy_j(m_kg):
    c = 2.99792458e8
    return m_kg * c * c
"""
with open("demo_bytecode.py", "w") as f:
    _ = f.write(src)
    
import py_compile, importlib.util, dis, binascii, sys, os
pyc_path = py_compile.compile("demo_bytecode.py", cfile=None, doraise=True) # write .pyc file
print("pyc written:", os.path.basename(pyc_path))
spec = importlib.util.spec_from_file_location("demo_bytecode", "demo_bytecode.py") # load module from source
mod = importlib.util.module_from_spec(spec); spec.loader.exec_module(mod) # execute module
print(mod.energy_j(0.10))
print(dis.code_info(mod.energy_j).splitlines()[0]) # first line of code info in pycodeobject
disassembled = dis.dis(mod.energy_j) # disassemble bytecode to human-readable form
print(disassembled)
with open(pyc_path, "rb") as f:
    head = f.read(24) # read first 24 bytes of .pyc file header
print("pyc header (hex):", binascii.hexlify(head).decode()) # header in hex
print("MAGIC_NUMBER:", binascii.hexlify(importlib.util.MAGIC_NUMBER).decode()) # which matches the first 4 bytes of the header, confirming Python version compatibility


## (Optional) 10. Programming foundations — C source and assembly (mock text files)

In [ ]:
# --- step 1: write a simple C source file and mock assembly output ---
# --- step 2: write mock assembly output as text file ---
# simple_energy.c: compute E=mc^2 for m=0.1 kg, c=speed of light, print result
# Compile with: cc -O2 -o energy simple_energy.c
# Disassemble with: objdump -d energy > simple_energy.s.txt
# Note: actual assembly and machine code will vary by system and compiler
c_src = """/* simple_energy.c (illustration) */
#include <stdio.h>
int main(void){
    const double c = 2.99792458e8;
    double m = 0.10;
    double E = m * c * c;
    printf("E = %.3e J\n", E);
    return 0;
}
"""
with open("simple_energy.c", "w") as f:
    _ = f.write(c_src)
print("Wrote simple_energy.c (compile with: cc -O2 -o energy simple_energy.c)")
asm_text = """; simple_energy.s (mock x86-64-ish assembly, illustrative)
    .globl main
main:
    push    rbp
    mov     rbp, rsp
    ; ... load constants, multiply, call printf ...
    mov     eax, 0
    pop     rbp
    ret
"""
with open("simple_energy.s.txt", "w") as f:
    _ = f.write(asm_text)
print("Wrote simple_energy.s.txt (mock assembly text)")

# --- Mock assembler+linker output as hex bytes (text file) ---
bytes_text = """; simple_energy.bytes.txt (mock machine-code bytes as if in an .exe)
; NOTE: this is illustrative hex, not a runnable binary

# .text (code)
55 48 89 E5                         # push rbp; mov rbp,rsp
F2 0F 10 05 F4 0F 00 00             # movsd xmm0, [rip+0xFF4]  -> points to 0x2000
F2 0F 10 0D F4 0F 00 00             # movsd xmm1, [rip+0xFF4]  -> points to 0x2008
F2 0F 59 C0                         # mulsd xmm0, xmm0
F2 0F 59 C1                         # mulsd xmm0, xmm1
F2 0F 11 45 F8                      # movsd [rbp-0x8], xmm0
B8 00 00 00 00                      # mov eax, 0
5D                                  # pop rbp
C3                                  # ret

# .rodata (constants; IEEE-754 little-endian doubles)
00 00 00 4A 78 DE B1 41             # 2.99792458e8
9A 99 99 99 99 99 B9 3F             # 0.1
"""
with open("simple_energy.bytes.txt", "w") as f:
    _ = f.write(bytes_text)  # write mock machine-code bytes (text)
print("Wrote simple_energy.bytes.txt (mock machine-code bytes)")

## 11. Python vs C — execution model, GIL note, and NumPy speed sketch

In [ ]:
# performance comparison: pure Python vs. NumPy for sum of squares
print("CPython uses a Global Interpreter Lock (GIL) to protect interpreter state;")
print("CPU-bound Python threads do not run in true parallel; use processes or vectorized libs.")
def py_sum_squares(n):
    s = 0
    for k in range(n):
        s = s + k*k
    return s
N = 2_000_000
import time
t0 = time.perf_counter()
_ = py_sum_squares(N)
t1 = time.perf_counter()
print("pure Python seconds:", round(t1 - t0, 4))
try:
    import numpy as np
    t2 = time.perf_counter()
    arr = np.arange(N, dtype=np.int64)
    _ = int((arr*arr).sum())
    t3 = time.perf_counter()
    print("NumPy seconds:", round(t3 - t2, 4))
except Exception as e:
    print("NumPy not available for timing:", e)


In [ ]:
# performance comparison: single vs two threads for CPU-bound and I/O-bound tasks
# Same task per thread, compare total time for single vs two threads
from concurrent.futures import ThreadPoolExecutor

# --- helpers ---
def py_sum_squares(n):                     # pure Python loop (CPU-bound)
    s = 0
    for k in range(n):
        s = s + k*k
    return s

def fake_io(seconds):                      # simulates blocking I/O (sleep releases the GIL)
    time.sleep(seconds)
    return seconds

N = 5_000_000  # adjust for your machine; increase to make CPU work visible
S = 1.0       # seconds of "I/O" per task

print("## CPU-bound: single vs two threads")
t0 = time.perf_counter()
_ = py_sum_squares(N)
t1 = time.perf_counter()
print("single thread (CPU)   :", round(t1 - t0, 3), "s")

# run two CPU tasks concurrently with threads
t2 = time.perf_counter()
with ThreadPoolExecutor(max_workers=2) as ex:
    list(ex.map(py_sum_squares, [N, N]))   # start two CPU-heavy tasks
t3 = time.perf_counter()
print("two threads (CPU)     :", round(t3 - t2, 3), "s  # expect ~2× single (no speedup)\n")

print("## I/O-bound: single vs two threads (sleep simulates I/O)")
t4 = time.perf_counter()
_ = fake_io(S)
t5 = time.perf_counter()
print("single thread (I/O)   :", round(t5 - t4, 3), "s")

t6 = time.perf_counter()
with ThreadPoolExecutor(max_workers=2) as ex:
    list(ex.map(fake_io, [S, S]))          # start two I/O waits together
t7 = time.perf_counter()
print("two threads (I/O)     :", round(t7 - t6, 3), "s  # expect ~S, near 2× throughput\n")

print("Note: Threads are used only as a small demo; no parallel speedup for CPU-bound Python due to the GIL.")


In [ ]:
# performance comparison: single vs two threads for CPU-bound and I/O-bound tasks
# Same overall task, compare total time for single vs two threads -- Different throughputs
def cpu_half(n):  # Half of the CPU work
    s = 0
    for k in range(n):
        s += k*k
    return s

def io_half(t):   # Half of the I/O work
    time.sleep(t)
    return t

N = 2_000_000     # enough to see effect
S = 2.0           # seconds total simulated I/O

# --- CPU-bound: split one big job into two halves ---
print("## CPU-bound")
t0 = time.perf_counter()
cpu_half(N*2)                                 # one thread does all
t1 = time.perf_counter()
with ThreadPoolExecutor(max_workers=2) as ex:
    list(ex.map(cpu_half, [N, N]))            # two halves in threads
t2 = time.perf_counter()
print("Single thread:", round(t1 - t0, 3), "s")
print("Two threads  :", round(t2 - t1, 3), "s  # no speedup due to GIL\n")

# --- I/O-bound: split one big job into two halves ---
print("## I/O-bound")
t3 = time.perf_counter()
io_half(S)                                    # one thread waits total S seconds
t4 = time.perf_counter()
with ThreadPoolExecutor(max_workers=2) as ex:
    list(ex.map(io_half, [S/2, S/2]))         # two waits overlap
t5 = time.perf_counter()
print("Single thread:", round(t4 - t3, 3), "s")
print("Two threads  :", round(t5 - t4, 3), "s  # ~2× speedup because waits overlap\n")
